# SQL Analysis on A/B Testing Campaigns

## Preparation

### Loading the Libraries and intalling packeges

 Installing pandasql

In [27]:
#!pip install pandasql

Importing libraries

In [28]:
import pandas as pd
from pandasql import sqldf

helper to run SQL against pandas DataFrames

In [29]:
pysqldf = lambda q: sqldf(q, globals())

Loading the CSV directly from GitHub

In [30]:
url = 'https://raw.githubusercontent.com/Adi-Shalit/AB-Testing-SQL-Python-Tableau-Project/main/AB_data_for_Analysis.csv'
campaign_data = pd.read_csv(url)

### Checking df

In [31]:
campaign_data.head()

,Campaign,Date,Spend_USD,Impressions,Reach,Website_Clicks,Searches,View_Content,Add_to_Cart,Purchases
0,Control Campaign,2019-08-01,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2019-08-02,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,2019-08-03,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,2019-08-04,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,2019-08-06,3083,109076.0,87998.0,4028.0,1709.0,1249.0,784.0,764.0


## SQL Analysis

### Campaign Performance Totals

In [32]:
query = """
    SELECT
      Campaign,
      SUM(Purchases) AS total_Purchases,
      SUM(Spend_USD) AS total_spend,
      SUM(Impressions) AS total_impressions,
      SUM(Reach) AS total_reach,
      SUM(Website_Clicks) AS total_Website_Clicks,
      SUM(Searches) AS total_Searches,
      SUM(View_Content) AS total_View_Content,
      SUM(Add_to_Cart) AS total_Add_to_Cart
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result


,Campaign,total_Purchases,total_spend,total_impressions,total_reach,total_Website_Clicks,total_Searches,total_View_Content,total_Add_to_Cart
0,Control Campaign,15161.0,66818,3177233.0,2576503.0,154303.0,64418.0,56370.0,37700.0
1,Test Campaign,15637.0,76892,2237544.0,1604747.0,180970.0,72569.0,55740.0,26446.0


* The Test Campaign generated more purchases (15,637) than the Control Campaign (15,161), even though the Control had more impressions.

* The Control Campaign had significantly more impressions (3.17M) and reach (2.58M) compared to the Test Campaign (2.24M impressions and 1.6M reach).

* The Test Campaign drove more website clicks (180,970) than the Control Campaign (154,303), This might suggest differences in effectiveness, but a proper CTR (Click-Through Rate) calculation is needed.

* The Test Campaign also generated more searches (72,569 vs. 64,418), indicating users were more likely to explore after clicking.

* Both campaigns had a similar number of “View Content” actions (~56K), despite the difference in clicks and reach.

* The Control Campaign had more “Add to Cart” actions (37,700 vs. 26,446), which may suggest higher purchase intent at that stage.

* The Test Campaign spent more budget overall (76,892 vs. 66,818), but with similar purchase outcomes, this might imply a lower return on ad spend.

### Average Reach, Website Clicks and Impressions

In [33]:
query = """
    SELECT
      Campaign,
      ROUND(AVG(Reach), 2) AS avg_reach,
      ROUND(AVG(Website_Clicks), 2) AS avg_Website_Clicks,
      ROUND(AVG(Impressions), 2) AS avg_impressions
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result


,Campaign,avg_reach,avg_Website_Clicks,avg_impressions
0,Control Campaign,88844.93,5320.79,109559.76
1,Test Campaign,53491.57,6032.33,74584.80


The Control Campaign reached a significantly larger audience on average, with higher impressions per day. However, despite its smaller reach, the Test Campaign generated more website clicks on average, suggesting better engagement per impression.

### Cost efficiency

In [34]:
query = """
    SELECT
      Campaign,
      SUM(Spend_USD) AS total_spend,
      SUM(Purchases) AS total_purchases,
      ROUND(SUM(Spend_USD) / SUM(Purchases), 2) AS cost_per_purchase
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result

,Campaign,total_spend,total_purchases,cost_per_purchase
0,Control Campaign,66818,15161.0,4.41
1,Test Campaign,76892,15637.0,4.92


Although the Test Campaign had slightly more purchases, the cost per purchase remains slightly lower for the Control Campaign (4.41 vs. 4.92), suggesting better cost efficiency even if fewer purchases are made overall.

### Campaign Conversion rate

In [35]:
query = """
    SELECT
      Campaign,
      SUM(Purchases) AS total_purchases,
      SUM(Website_Clicks) AS total_website_clicks,
      ROUND(SUM(Purchases) / SUM(Website_Clicks), 4) AS conversion_rate
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result

,Campaign,total_purchases,total_website_clicks,conversion_rate
0,Control Campaign,15161.0,154303.0,0.0983
1,Test Campaign,15637.0,180970.0,0.0864


The Control Campaign shows a slightly higher conversion rate from clicks to purchases (9.83%) than the Test Campaign (8.64%), indicating it was slightly better at turning interest into action.

A conversion rate of around 8% is generally considered quite strong for most digital marketing campaigns.

This suggests that a significant portion of users who clicked through to the website ended up making a purchase, indicating that the campaigns were effective at driving qualified traffic.

### Campaign Efficiency Metrics - different rates between each step in the funnel

In [36]:
query = """
    SELECT
      Campaign,
      ROUND(SUM(Website_Clicks) / SUM(Impressions), 4) AS CTR,
      ROUND(SUM(Add_to_Cart) / SUM(View_Content), 4) AS add_to_cart_rate,
      ROUND(SUM(Purchases) / SUM(Add_to_Cart), 4) AS purchase_rate,
      ROUND(SUM(View_Content) / SUM(Website_Clicks), 4) AS view_content_rate
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result

,Campaign,CTR,add_to_cart_rate,purchase_rate,view_content_rate
0,Control Campaign,0.0486,0.6688,0.4021,0.3653
1,Test Campaign,0.0809,0.4745,0.5913,0.3080


* The Test Campaign now leads in Click-Through Rate (CTR) with 0.0809 compared to 0.0486 for the Control Campaign, indicating it's more successful at driving traffic from impressions to clicks.

* The Control Campaign outperforms significantly in add-to-cart rate (66.88% vs. 47.45%), suggesting that users who engage with it are more inclined to take the next step toward purchasing.

* Interestingly, the Test Campaign shows a much higher purchase rate from add-to-cart (59.13% vs. 40.21%), pointing to stronger intent or more effective checkout flows among those who reach this stage.

* The Control Campaign also leads in view-content rate (36.53% vs. 30.80%), implying that users clicking through are more consistently exploring product details.

### Number of days recorded

In [37]:
query = """
    SELECT
      Campaign,
      SUBSTR(Date, 1, 4) AS Year,
      SUBSTR(Date, 6, 2) AS Month,
      COUNT(*) AS days_count
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result

,Campaign,Year,Month,days_count
0,Control Campaign,2019,08,29
1,Test Campaign,2019,08,30


29 days recorded for the control campaign, and similarly 30 days for the Test campaign, all in August 2019.

### Maximum daily impressions

In [38]:
query = """
    SELECT
      Campaign,
      MAX(Impressions) AS max_impressions,
      Date
    FROM campaign_data
    GROUP BY Campaign;
"""
result = pysqldf(query)
result

,Campaign,max_impressions,Date
0,Control Campaign,145248.0,2019-08-14
1,Test Campaign,133771.0,2019-08-21


### Days where Test campaign outperformed Control in conversion rate

In [39]:
query = """
    WITH daily_rates AS (
      SELECT Campaign, Date, Purchases / Website_Clicks AS conv_rate
      FROM campaign_data
    )
    SELECT
      t.Date,
      t.conv_rate AS test_rate,
      c.conv_rate AS control_rate,
      ROUND((t.conv_rate - c.conv_rate), 4) AS rate_diff
    FROM daily_rates t
    JOIN daily_rates c ON t.Date = c.Date
    WHERE t.Campaign = 'Test Campaign'
      AND c.Campaign = 'Control Campaign'
    ORDER BY rate_diff DESC
    LIMIT 13;
"""
result = pysqldf(query)
result

,Date,test_rate,control_rate,rate_diff
0,2019-08-11,0.178133,0.058375,0.1198
1,2019-08-25,0.181034,0.091167,0.0899
2,2019-08-02,0.145373,0.063009,0.0824
3,2019-08-29,0.088485,0.041098,0.0474
4,2019-08-26,0.074346,0.046182,0.0282
5,2019-08-07,0.210851,0.189015,0.0218
6,2019-08-09,0.102313,0.080833,0.0215
7,2019-08-03,0.073304,0.057160,0.0161
8,2019-08-22,0.062288,0.056554,0.0057
9,2019-08-15,0.079712,0.074755,0.0050


On specific days, the Test Campaign significantly outperformed the Control Campaign in conversion rate.
For example, on August 11th, the Test Campaign achieved a conversion rate of 17.8%, compared to just 5.8% for the Control Campaign - a difference of nearly 12 percentage points.

Although there are a days where the Test Campaign had a higher conversion rate than the Control Campaign, these are not the majority of cases, the rate diff is negative on the 13th row (number 12), meaning the Test Campaign had a higher conversion rate 12 days out of 30.

Given that the dataset spans 30 days, and only a 12 days show the Test Campaign outperforming, it suggests that overall, the Control Campaign maintained stronger and more consistent performance in driving purchases from clicks.